# Import Package

In [18]:
#dataframe
import pandas as pd

#computation
import numpy as np

#NLP lib
import nltk

# string operations
import string 
import re

# general imports
import math


#visualization
import matplotlib.pyplot as plt

#others
import collections
from collections import Counter

#pre-build package
from utils import *
from Invertedfile_TFIDF import Invertedfile_TFIDF
from AverageWordEmbedding import AverageWordEmbedding
from WeightedAverageEmbedding import WeightedAverageEmbedding

#auto reload
%load_ext autoreload
%autoreload 2

#ignore warning
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip

# Load Data

In [19]:
df = pd.read_csv('data.tsv', sep='\t',error_bad_lines=False)
df.head()

b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'


,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0


In [20]:
df.dropna(inplace=True)
#drop rows with dupplication
df = df.drop_duplicates(subset = ['question1'], keep = 'last')
df = df.drop_duplicates(subset = ['question2'], keep = 'last')
df.nunique()

id              232203
qid1            232203
qid2            232202
question1       232203
question2       232203
is_duplicate         2
dtype: int64

### Test Set

In [21]:
df.reset_index(inplace=True)

In [22]:
#Get the first 100 "is duplicate" rows for testing
df_test = df[df['is_duplicate'] == 1].iloc[:100]
df_test.head()

,index,id,qid1,qid2,question1,question2,is_duplicate
1,1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
6,13,11568,22332,22333.0,Which is the best book to study TENSOR for gen...,Which is the best book for tensor calculus?,1.0
11,24,33995,62359,62360.0,How does an IQ test work and what is determine...,How does IQ test works?,1.0
12,25,244506,357159,357160.0,Is it safe to use Xiaomi mobile phones?,Is it safe or unsafe to use Xiaomi Products?,1.0
14,27,375073,506056,506057.0,What are the best books on cosmology?,Which is the best book for cosmology?,1.0


# Text Preprocessing

In [23]:
df['processed_ques2'] = df.question2.apply(lambda x: text_preprocessing(x))

# TFIDF with Inverted File

In [33]:
model1 = Invertedfile_TFIDF(df[:30000])

Build vocabulary...
Build TF-IDF matrix...
Build inverted file...
Model is ready to query!


In [34]:
top2  = model1.accuracy(df_test, 2) 
top5  = model1.accuracy(df_test, 5) 

print("Accuracy with k = 2: ", top2)
print("Accuracy wit k = 5: ", top5)

Accuracy with k = 2:  0.75
Accuracy wit k = 5:  0.91


In [39]:
model1.query_engine()

Your query is: How to enjoy life
 How do you enjoy life as a freelancer? - 5.179750382181805
-----------------------------------------------------
 Do you enjoy work? Why? - 4.87169593166282
-----------------------------------------------------
 How much do you enjoy cricket? - 3.247797287775213
-----------------------------------------------------
 How much do you enjoy basketball? - 3.247797287775213
-----------------------------------------------------
 What will you do if you fail in your life? - 2.8979296416098874
-----------------------------------------------------


# Averaging Word Embedding

In [35]:
model2 = AverageWordEmbedding('glove.6B.100d.txt', 100, df[:30000])

Loading Glove Model...
Embedding All Question 2 data...
Model is ready to query


In [36]:
top2  = model2.accuracy(df_test, 2) 
top5  = model2.accuracy(df_test, 5) 

print("Accuracy with k = 2: ", top2)
print("Accuracy wit k = 5: ", top5)

Accuracy with k = 2:  0.81
Accuracy wit k = 5:  0.85


In [40]:
model2.query_engine()

Your query is: How to learn faster
 How can I remember faster? - -1.9246873809513352
-----------------------------------------------------
 What is the best way to learn and understand C++ faster? - -2.2462644407355805
-----------------------------------------------------
 How can I learn kannada faster? - -2.5424158592322144
-----------------------------------------------------
 How can I train my brain to think faster? - -2.5925073400224883
-----------------------------------------------------
 How do I teach myself to learn better? - -2.594176584259434
-----------------------------------------------------


# TFIDF Weighted Average Word Embedding

In [37]:
model3 = WeightedAverageEmbedding('glove.6B.100d.txt', 100, df[:30000])

Build vocabulary...
Loading Glove Model...
Embedding All Question 2 data...
Model is ready to query


In [38]:
top2  = model3.accuracy(df_test, 2) 
top5  = model3.accuracy(df_test, 5) 

print("Accuracy with k = 2: ", top2)
print("Accuracy wit k = 5: ", top5)

Accuracy with k = 2:  0.64
Accuracy wit k = 5:  0.65


In [41]:
model3.query_engine()

Your query is: How to stay motivated
 How have I motivated others? - -2.070999478078923
-----------------------------------------------------
 Am I justified in being afraid to bring a child into the world? - -2.478648188885372
-----------------------------------------------------
 Is it normal to have cheated in a relationship even though you wouldn't tolerate that behavior from your partner? - -2.79645873615929
-----------------------------------------------------
 I am a student and feeling annoyed to my family because of their behaviour. I can't focus on my studies. Suggestion? - -2.8029969546267064
-----------------------------------------------------
 What are some good patterns from the past that tell us that countries are likely to engage in a war? - -2.8677332188447213
-----------------------------------------------------
